# Do some imports

In [1]:
!pip install --user pandas
!pip install --user scikit-learn
!pip install tqdm

from typing import List
import numpy as np
import pandas as pd
#

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

from brevitas.nn import QuantIdentity, QuantConv2d, QuantReLU, QuantLinear, QuantHardTanh
from brevitas.core.quant import QuantType

from dataloader import UNSW_NB15
from dataloader_quantized import UNSW_NB15_quantized

     |################################| 9.5MB 2.1MB/s eta 0:00:01     |#######################         | 6.8MB 2.1MB/s eta 0:00:02
     |################################| 6.8MB 4.5MB/s eta 0:00:01     |#########                       | 2.1MB 183kB/s eta 0:00:26     |##############                  | 3.2MB 183kB/s eta 0:00:20     |############################    | 6.1MB 4.5MB/s eta 0:00:01
     |################################| 307kB 10.3MB/s eta 0:00:01


### Inspired by [this github file](https://github.com/alik604/cyber-security/blob/master/Intrusion-Detection/UNSW_NB15%20-%20Torch%20MLP%20and%20autoEncoder.ipynb)

# Get UNSW_NB15 train and test set

In [2]:
#!wget https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/a%20part%20of%20training%20and%20testing%20set/UNSW_NB15_training-set.csv

In [3]:
#!wget https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/a%20part%20of%20training%20and%20testing%20set/UNSW_NB15_testing-set.csv

# Define the Neural Network class

### Define Train,   Test   and    Display_Loss_Plot    methods

In [62]:
def train(model, device, train_loader, optimizer, criterion):
    losses = []
    model.train()
    y_true = []
    y_pred = []
    
    for i, data in enumerate(train_loader, 0):        
        # get the inputs; data is a list of [inputs, target ( or labels)]
        inputs , target = data
        optimizer.zero_grad()   
        
        #FORWARD PASS
        output = model(inputs.float())
        print("Check if output has nans=",torch.isnan(output).any())

        loss = criterion(output, target.unsqueeze(1))
        
        #BACKWARD AND OPTIMIZE        
        loss.backward()
        optimizer.step()
        
        # PREDICTIONS
        #pred = np.round(output.detach().numpy())
        pred = output.detach().numpy() > 0.5  
        target = target.float()
        y_true.extend(target.tolist()) 
        y_pred.extend(pred.reshape(-1).tolist())
        
        losses.append(loss.data.numpy()) 
    #print("Accuracy on training set is" , accuracy_score(y_true,y_pred))
    return losses

In [65]:
#TESTING THE MODEL
def test(model, device, test_loader):    
    model.eval()   #model in eval mode skips Dropout etc
    y_true = []
    y_pred = []
   
    with torch.no_grad(): # set the requires_grad flag to false as we are in the test mode
        for data in test_loader:
            
            #LOAD THE DATA IN A BATCH
            inputs ,target = data
            
            # the model on the data
            output = model(inputs.float())
            print("Check if output has nans=",torch.isnan(output).any())

            output = nn.Sigmoid(output)  
            
            #PREDICTIONS
            pred = np.round(output)
            #pred = output.detach().numpy() > 0.5 
            pred = pred * 1
            target = target.float()
            y_true.extend(target.tolist()) 
            y_pred.extend(pred.reshape(-1).tolist())
        
    return accuracy_score(y_true, y_pred)

In [41]:
def display_loss_plot(losses):
    x_axis = [i for i in range(len(losses))]
    plt.plot(x_axis,losses)
    plt.title('Loss of the model')
    plt.xlabel('iterations')
    plt.ylabel('Cross entropy loss')
    plt.show()

# Define some parameters first

In [60]:
device = 'cpu'
input_size = 196      # 42 for integer encoding 196
hidden1 = 128      # 1st layer number of neurons
hidden2 = 64
hidden3 = 32
num_classes = 1    # binary classification

num_epochs = 2
batch_size = 5000 
lr = 0.001        

# Initialize Neural Network class

In [56]:
class QuantLeNet(nn.Module):
    def __init__(self, input_size,hidden1, hidden2, hidden3, num_classes):
        super(QuantLeNet, self).__init__()
        self.fc1   = QuantLinear(input_size, hidden1, bias=True, weight_bit_width=8)
        self.batchnorm1 = nn.BatchNorm1d(hidden1)
        self.relu1 = QuantReLU(bit_width=8)
        
        self.fc2   = QuantLinear(hidden1, hidden2, bias=True, weight_bit_width=8)
        self.batchnorm2 = nn.BatchNorm1d(hidden2)
        self.relu2 = QuantReLU(bit_width=8)
        
        self.fc3   = QuantLinear(hidden2, hidden3, bias=True, weight_bit_width=8)
        self.batchnorm3 = nn.BatchNorm1d(hidden3)
        self.relu3 = QuantReLU(bit_width=8)
        
        self.out   = QuantLinear(hidden3, num_classes, bias=False, weight_bit_width=8)
        self.batchnorm4 = nn.BatchNorm1d(num_classes)
        #self.relu4 = QuantReLU(bit_width=1, max_val=1)
        

    def forward(self, x):
        out = self.fc1(x)
        out = self.batchnorm1(out)
        out = self.relu1(out)
        
        out = self.fc2(out)
        out = self.batchnorm2(out)
        out = self.relu2(out)
        
        out = self.fc3(out)
        out = self.batchnorm3(out)
        out = self.relu3(out)
        
        out = self.out(out)
        out = self.batchnorm4(out)
        return out
    
model = QuantLeNet(input_size, hidden1, hidden2, hidden3, num_classes)

# Define loss and optimizer 

In [57]:
criterion = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

# Initialize UNSW_NB15 class

In [20]:
#these are not slitted into validation and train set
train_dataset = UNSW_NB15(file_path ='data/UNSW_NB15_training-set.csv')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#get the test dataframe
test_dataset = UNSW_NB15(file_path ='data/UNSW_NB15_testing-set.csv')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

#Get the Quantized versions
train_quantized_dataset = UNSW_NB15_quantized(file_path_train='data/UNSW_NB15_training-set.csv', \
                                              file_path_test = "data/UNSW_NB15_testing-set.csv", \
                                              train=True)
train_quantized_loader = DataLoader(train_quantized_dataset, batch_size=batch_size, shuffle=True)

test_quantized_dataset = UNSW_NB15_quantized(file_path_train='data/UNSW_NB15_training-set.csv', \
                                              file_path_test = "data/UNSW_NB15_testing-set.csv", \
                                              train=False)
test_quantized_loader = DataLoader(test_quantized_dataset, batch_size=batch_size, shuffle=True)

torch.Size([175341, 197])
torch.Size([82332, 197])
torch.Size([175341, 197])
torch.Size([82332, 197])


# Lets Train, Test the model and see the loss

In [58]:
train_loader = train_quantized_loader
test_loader = test_quantized_loader

In [63]:
running_loss = []
for epoch in tqdm(range(num_epochs)):
        loss_epoch = train(model, device, train_loader, optimizer,criterion)
        running_loss.append(loss_epoch)
#Save the model!!
torch.save(model.state_dict(), "MLP_model")






  0%|          | 0/2 [00:00<?, ?it/s]

Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if o






 50%|█████     | 1/2 [00:10<00:10, 10.94s/it]

Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)
Check if o






100%|██████████| 2/2 [00:21<00:00, 10.73s/it]

Check if output has nans= tensor(1, dtype=torch.uint8)
Check if output has nans= tensor(1, dtype=torch.uint8)


In [66]:
#model.load_state_dict(torch.load("MLP_model"))
test(model,device,test_loader)

Check if output has nans= tensor(1, dtype=torch.uint8)


TypeError: __init__() takes 1 positional argument but 2 were given

In [ ]:
loss_per_epoch = [np.mean(loss_per_epoch) for loss_per_epoch in running_loss]
display_loss_plot(loss_per_epoch)

**********************************************************************************************************************

# Results

    num_epochs = 200
    batch_size = 8000 
    lr = 0.001
    accuracy on test set = 0.7235218384103387
<img src="data/train_not_quantized_72.PNG">